In [1]:
import numpy as np
import autokeras as ak
import tensorflow as tf
from tensorflow.keras.models import load_model

2023-05-11 22:45:23.757578: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64
2023-05-11 22:45:23.757595: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
model = load_model('clip_autokeras_nsfw_b32', custom_objects=ak.CUSTOM_OBJECTS)

2023-05-11 22:45:52.358871: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-11 22:45:52.359114: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64
2023-05-11 22:45:52.359189: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64
2023-05-11 22:45:52.359234: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PA

In [17]:
model.layers[7].rate

0.5

In [19]:
model.layers

In [20]:
from torch.nn import functional as F
import torch
from torch import nn

torch.manual_seed(8492)
embed = F.normalize(torch.randn([4, 512])).numpy()
real_model_out = model.predict(embed)
real_model_out

array([[4.6667284e-12],
       [1.4484495e-01],
       [3.4860303e-12],
       [1.3105124e-02]], dtype=float32)

In [21]:
for i, layer in enumerate(model.layers):
    for j, weight in enumerate(layer.weights):
        if hasattr(weight, 'shape'):
            print(i, j, weight.shape)

2 0 (512,)
2 1 (512,)
2 2 ()
3 0 (512, 32)
3 1 (32,)
5 0 (32, 32)
5 1 (32,)
8 0 (32, 1)
8 1 (1,)


In [22]:
class Normalization(nn.Module):
    def __init__(self, shape):
        super().__init__()
        self.register_buffer('mean', torch.zeros(shape))
        self.register_buffer('variance', torch.ones(shape))

    def forward(self, x):
        return (x - self.mean) / self.variance.sqrt()
    

class NSFWModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.norm = Normalization([512])
        self.linear_1 = nn.Linear(512, 32)
        self.linear_2 = nn.Linear(32, 32)
        self.linear_3 = nn.Linear(32, 1)
        self.dropout = nn.Dropout(0.5)
        self.act = nn.ReLU()
        self.act_out = nn.Sigmoid()

    def forward(self, x):
        x = self.norm(x)
        x = self.act(self.linear_1(x))
        x = self.dropout(self.act(self.linear_2(x)))
        x = self.act_out(self.linear_3(x))
        return x

In [23]:
pt_model = NSFWModel()

In [24]:
pt_model.state_dict().keys()

odict_keys(['norm.mean', 'norm.variance', 'linear_1.weight', 'linear_1.bias', 'linear_2.weight', 'linear_2.bias', 'linear_3.weight', 'linear_3.bias'])

In [26]:
from collections import OrderedDict

state_dict = OrderedDict()
state_dict['norm.mean'] = torch.tensor(model.layers[2].weights[0].numpy())
state_dict['norm.variance'] = torch.tensor(model.layers[2].weights[1].numpy())
state_dict['linear_1.weight'] = torch.tensor(model.layers[3].weights[0].numpy()).T
state_dict['linear_1.bias'] = torch.tensor(model.layers[3].weights[1].numpy())
state_dict['linear_2.weight'] = torch.tensor(model.layers[5].weights[0].numpy()).T
state_dict['linear_2.bias'] = torch.tensor(model.layers[5].weights[1].numpy())
state_dict['linear_3.weight'] = torch.tensor(model.layers[8].weights[0].numpy()).T
state_dict['linear_3.bias'] = torch.tensor(model.layers[8].weights[1].numpy())

pt_model.load_state_dict(state_dict)

<All keys matched successfully>

In [27]:
pt_model(torch.tensor(embed)).detach().numpy() - real_model_out

array([[ 1.2043035e-09],
       [ 1.5906852e-01],
       [-3.4599266e-12],
       [-6.2316759e-03]], dtype=float32)

In [28]:
torch.save(pt_model.state_dict(), 'clip_autokeras_binary_nsfw_b32.pth')